# Compiling and Submitting Circuits onto Hilbert via Cirq


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Infleqtion/client-superstaq/blob/main/docs/source/optimizations/hilbert/hilbert_compile_css.ipynb) [![Launch Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Infleqtion/client-superstaq/HEAD?labpath=docs/source/optimizations/hilbert/hilbert_compile_css.ipynb)

## Import Requirements

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the ```cirq-superstaq``` client. You can try it out by running ```pip install cirq-superstaq[examples]```:

In [1]:
# Required imports
try:
    import cirq
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq
    import cirq_superstaq as css

# Optional imports
import os  # Used if setting a token as an environment variable

To interface Superstaq via Cirq, we must first instantiate a service provider in ```cirq-superstaq``` with ```Service()```. We then supply a Superstaq API key (which you can get from https://superstaq.infleqtion.com) by either providing the API key as an argument of Service, i.e., ```css.Service(api_key="token")```, or by setting it as an environment variable. (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
service = css.Service()

## Create a Circuit

First, we will create an example Cirq circuit that we can then compile and submit onto Hilbert

In [3]:
qubits = cirq.LineQubit.range(4)
circuit1 = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(qubits[0]))
print(circuit1)

0: ───H───@───M───
          │
1: ───────X───────


## Single Circuit Compilation

We will now compile the above circuit onto Hilbert's native gateset and visualize the differences by drawing the compiled circuit

In [4]:
compiler_output = service.cq_compile(circuit1)

In [5]:
print(compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────RGate(0.25π, -0.5π)───────M('q(0)')───
           │                    │   │                          │
(0, 1): ───#2───────────────────@───#2─────────────────────Z───#2────────────────────Z───────────────


If you would like to compile (or submit) on a different number of qubits, this can be done via the ```grid_shape``` option. This simply sets the shape of the rectangular qubit grid. However, specifying a grid that is incompatible with Hilbert's current capabilities will result in an error when submitting.

In [6]:
example_circuit = cirq.Circuit(cirq.H.on_each(*qubits))
new_compiler_output = service.cq_compile(example_circuit, grid_shape=(4, 1))
print(new_compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───Z───
           │
(1, 0): ───#2───────────────────Z───
           │
(2, 0): ───#3───────────────────Z───
           │
(3, 0): ───#4───────────────────Z───


## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in job_css from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [7]:
job_css = service.create_job(circuit=circuit1, repetitions=100, target="cq_hilbert_simulator")
result_css = job_css.counts()
print(f"Counts from cirq-superstaq submission: {result_css}")

Counts from cirq-superstaq submission: {'0': 54, '1': 46}


## Multiple circuit compilation

All the functionalities we have seen so far can also be used on a multiple-circuit input as well. To illustrate this, let us create a different example two-qubit circuit, and compile both circuits we have created at the same time. 

In [8]:
circuit2 = cirq.Circuit(cirq.H(qubits[0]), cirq.measure(qubits[0]))
print(circuit2)

0: ───H───M───


In [9]:
compiler_output = service.cq_compile([circuit1, circuit2])

In [10]:
print(compiler_output.circuits[0])
print()
print()
print(compiler_output.circuits[1])

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────RGate(0.25π, -0.5π)───────M('q(0)')───
           │                    │   │                          │
(0, 1): ───#2───────────────────@───#2─────────────────────Z───#2────────────────────Z───────────────


(0, 0): ───RGate(0.5π, -0.5π)───M('q(0)')───
